In [167]:
year_data = 2025

In [168]:
import requests
import pandas as pd

In [169]:
ergast_base_url = "https://api.jolpi.ca/ergast/f1/"
# https://github.com/jolpica/jolpica-f1/blob/main/docs/README.md

In [170]:
def get_drivers_df(year):
    url = f"{ergast_base_url}/{year}/drivers/"
    response = requests.get(url)
    
    drivers_list = response.json()['MRData']['DriverTable']['Drivers']
    drivers_df = pd.json_normalize(drivers_list)
    
    # Rename columns
    drivers_df = drivers_df.rename(columns={'driverId': 'driver_id', 'permanentNumber': 'permanent_number', 'givenName': 'given_name'
        , 'familyName': 'family_name', 'dateOfBirth': 'birth_day'})
    
    # Assign types
    drivers_df = drivers_df.astype({'permanent_number': int, 'birth_day': 'datetime64[ns]'})
    
    return drivers_df

In [171]:
def get_constructors_df(year):
    url = f"{ergast_base_url}/{year}/constructors/"
    response = requests.get(url)
    
    constructors_list = response.json()['MRData']['ConstructorTable']['Constructors']
    constructors_df = pd.json_normalize(constructors_list)
    
    # Rename columns
    constructors_df = constructors_df.rename(columns={'constructorId': 'constructor_id'})
    
    return constructors_df

In [172]:
def get_races_calendar_df(year):
    url = f"{ergast_base_url}/{year}"
    response = requests.get(url)
    
    races_list = response.json()['MRData']['RaceTable']['Races']    
    races_df = pd.json_normalize(races_list)
    
    # Drop unnecessary columns
    races_df.drop(['FirstPractice.time', 'SecondPractice.time', 'ThirdPractice.time'], axis=1, inplace=True)
    
    # Rename columns
    races_df = races_df.rename(columns={'raceName': 'race_name', 'url': 'url_grand_prix'
        , 'Circuit.circuitId': 'circuit_id', 'Circuit.url': 'url_circuit'
        , 'Circuit.circuitName': 'name', 'Circuit.Location.lat': 'location_lat', 'Circuit.Location.long': 'location_long'
        , 'Circuit.Location.locality': 'location_locality', 'Circuit.Location.country': 'country'
        , 'FirstPractice.date': 'practice1_date', 'SecondPractice.date': 'practice2_date', 'ThirdPractice.date': 'practice3_date'
        , 'Qualifying.date': 'qualifying_date', 'Qualifying.time': 'qualifying_time'
        , 'Sprint.date': 'sprint_date', 'Sprint.time': 'sprint_time'
        , 'SprintQualifying.date': 'sprint_qualy_date', 'SprintQualifying.time': 'sprint_qualy_time'})
    
    # Assign types
    races_df = races_df.astype({'season': int, 'round': int, 'date': 'datetime64[ns]', 'location_lat': float, 'location_long': float
        , 'practice1_date': 'datetime64[ns]', 'practice2_date': 'datetime64[ns]', 'practice3_date': 'datetime64[ms]'
        , 'qualifying_date': 'datetime64[ns]', 'sprint_date': 'datetime64[ns]', 'sprint_qualy_date': 'datetime64[ns]'})
    
    return races_df

In [173]:
def get_races_results_df(year, year_races_calendar_df):
    # Start season races pd df
    races_df = pd.DataFrame()

    # Get season length
    season_length = len(year_races_calendar_df.index)

    # For each race
    for round_index in range(1,season_length+1):
        try:
            url = f"{ergast_base_url}/{year}/{round_index}/results"
            response = requests.get(url)
            
            # Get race result into a pd df
            race_result_df = pd.json_normalize(response.json()['MRData']['RaceTable']['Races'][0]['Results'])
            
            # Add circuit id, season year, round id
            race_row = year_races_calendar_df.loc[year_races_calendar_df['round'].astype(int) == round_index]

            race_result_df['raceRoundId'] = race_row['round']
            race_result_df['seasonYear'], race_result_df['circuitId'] = race_row['season'], race_row['circuit_id']

            # Add race result df to season races df
            races_df = pd.concat([races_df, race_result_df], ignore_index=True)
        except:
            # If no more races
            break 
    
    # Keep these columns 
    races_df = races_df[['number','position','positionText','points','grid','laps','status','Driver.permanentNumber','Driver.code'
        ,'Constructor.constructorId','Time.millis','Time.time','FastestLap.rank','FastestLap.lap','FastestLap.Time.time'
        ,'raceRoundId','seasonYear','circuitId']]
    
    # Rename columns
    races_df = races_df.rename(columns={'number': 'driver_number', 'positionText': 'position_text'
        , 'Driver.permanentNumber': 'driver_permanent_number', 'Driver.code': 'driver_code'
        , 'Constructor.constructorId': 'constructor_id', 'Time.millis': 'time_millis', 'Time.time': 'time_interval'
        , 'FastestLap.rank': 'fastest_lap_rank', 'FastestLap.lap': 'fastest_lap_lap', 'FastestLap.Time.time': 'fastest_lap_time'
        , 'raceRoundId': 'race_round_id', 'seasonYear': 'season', 'circuitId': 'race_circuit_id'})
    
    # Fill NaN
    races_df.fillna({'fastest_lap_rank': 0, 'fastest_lap_lap': 0, 'season': 0, 'race_round_id': 0}, inplace=True)
    
    # Assign types
    races_df = races_df.astype({'driver_number': int, 'position': int, 'points': int, 'grid': int, 'laps': int
        , 'driver_permanent_number': int, 'time_millis': float, 'fastest_lap_rank': int, 'fastest_lap_lap': int
        , 'race_round_id': int, 'season': int})
    
    return races_df

In [174]:
if __name__ == "__main__":
    year_drivers_df = get_drivers_df(year_data)
    year_constructors_df = get_constructors_df(year_data)
    year_races_calendar_df = get_races_calendar_df(year_data)
    #year_data_df = get_races_results_df(year_data, year_races_calendar_df)

In [175]:
#year_data_df.info()

In [ ]:
#print(year_drivers_df.to_markdown())
#print(year_constructors_df.to_markdown())
#print(year_races_calendar_df.to_markdown())

|    |   season |   round | url_grand_prix                                               | race_name                 | date                | time      | circuit_id    | url_circuit                                                          | name                           |   location_lat |   location_long | location_locality   | country      | practice1_date      | practice2_date      | practice3_date      | qualifying_date     | qualifying_time   | sprint_date         | sprint_time   | sprint_qualy_date   | sprint_qualy_time   |
|---:|---------:|--------:|:-------------------------------------------------------------|:--------------------------|:--------------------|:----------|:--------------|:---------------------------------------------------------------------|:-------------------------------|---------------:|----------------:|:--------------------|:-------------|:--------------------|:--------------------|:--------------------|:--------------------|:------------------|:------------

In [177]:
#print(year_data_df.to_markdown())